In [ ]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
EPOCHS = 5
STEP_SIZE = 0.1

In [ ]:
train_df = pd.read_csv('')
train_df = train_df.sample(frac=1)
train_df = train_df.fillna(train_df.median())
train_df = train_df.drop_duplicates()

In [ ]:
test_sizes = np.arange(0.1, 1.0, 0.1)

acci = []
for test_size in test_sizes:
    epoch_acci = []

    for epoch in range(EPOCHS):
        train_set, test_set = train_test_split(train_df, test_size=test_size, random_state=42)
        imputer = SimpleImputer(strategy='median')

        X_train = imputer.fit_transform(train_set[['eye_category', 'eye_position', 'gaze_direction', 'heart_rates', 'p2p_intervals', 'sys_peaks', 'dys_peaks']])
        X_test = imputer.transform(test_set[['eye_category', 'eye_position', 'gaze_direction', 'heart_rates', 'p2p_intervals', 'sys_peaks', 'dys_peaks']])

        y_train = train_set['engagement_labels']
        y_test = test_set['engagement_labels']

        early_fused_model = RandomForestClassifier(random_state=42)
        early_fused_model.fit(X_train, y_train)
        early_fused_test = early_fused_model.predict(X_test)
        accuracy = accuracy_score(y_test, early_fused_test)

        epoch_acci.append(accuracy)

    acci.append(sum(epoch_acci) / len(epoch_acci))

plt.figure(figsize=(12, 8))
plt.plot(test_sizes, acci)
plt.xlabel('Test Size')
plt.ylabel('Accuracy')
plt.show();